In [1]:
import cv2
import pickle

In [2]:
with open('model', 'rb') as f:
    svm = pickle.load(f)

In [3]:
names = {0 : 'No Mask', 1 : 'Mask'}

In [4]:
import screen_brightness_control as sbc
pre = sbc.get_brightness()
pre

20

In [5]:
sbc.set_brightness(100)
data = cv2.CascadeClassifier('data.xml')
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read(cv2.IMREAD_GRAYSCALE)
#     frame = change_brightness(frame, value=75)



#     frame = cv2.addWeighted(frame, 2, np.ones(frame.shape, frame.dtype), 0, 10)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if ret:
        face = data.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
        
        for x, y, w, h in face:
            cv2.rectangle(gray, (x, y), (x+w, y+h), (255, 255, 0), 2)
            face_array = gray[y:y+h, x:x+w]
            face_array = cv2.resize(face_array, (100, 100))
            face_array = face_array.reshape(1, -1)
#             face_array = pca.transform(face_array)
            pred = svm.predict(face_array)[0]
            n = names[int(pred)]
#             print(n)
            cv2.putText(gray, n, (x+15, y-10), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,0),2,cv2.LINE_AA)
        cv2.imshow('video', gray)
        
        if cv2.waitKey(1) & 0xff == 27:
            break
    
cap.release()
cv2.destroyAllWindows()
sbc.set_brightness(pre)